In [9]:
with open("Data/the-verdict.txt","r", encoding="utf-8") as f:
    raw_text=f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [11]:
import re

text = "Hello, World. This, is a test."
result= re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [13]:
result= re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'World', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [15]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'World', '.', 'This', ',', 'is', 'a', 'test', '.']


In [17]:
text= "Hello, world. Is this-- a test?"
result= re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [19]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed= [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [21]:
print(len(preprocessed))

4690


In [23]:
#Assing IDs to Tokens

In [25]:
all_words= sorted(set(preprocessed))
vocab_size=len(all_words)
print(vocab_size)

1130


In [27]:
vocab = {token: integer for integer,token in enumerate(all_words)}
print(vocab)

{'!': 0, '"': 1, "'": 2, '(': 3, ')': 4, ',': 5, '--': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'Ah': 12, 'Among': 13, 'And': 14, 'Are': 15, 'Arrt': 16, 'As': 17, 'At': 18, 'Be': 19, 'Begin': 20, 'Burlington': 21, 'But': 22, 'By': 23, 'Carlo': 24, 'Chicago': 25, 'Claude': 26, 'Come': 27, 'Croft': 28, 'Destroyed': 29, 'Devonshire': 30, 'Don': 31, 'Dubarry': 32, 'Emperors': 33, 'Florence': 34, 'For': 35, 'Gallery': 36, 'Gideon': 37, 'Gisburn': 38, 'Gisburns': 39, 'Grafton': 40, 'Greek': 41, 'Grindle': 42, 'Grindles': 43, 'HAD': 44, 'Had': 45, 'Hang': 46, 'Has': 47, 'He': 48, 'Her': 49, 'Hermia': 50, 'His': 51, 'How': 52, 'I': 53, 'If': 54, 'In': 55, 'It': 56, 'Jack': 57, 'Jove': 58, 'Just': 59, 'Lord': 60, 'Made': 61, 'Miss': 62, 'Money': 63, 'Monte': 64, 'Moon-dancers': 65, 'Mr': 66, 'Mrs': 67, 'My': 68, 'Never': 69, 'No': 70, 'Now': 71, 'Nutley': 72, 'Of': 73, 'Oh': 74, 'On': 75, 'Once': 76, 'Only': 77, 'Or': 78, 'Perhaps': 79, 'Poor': 80, 'Professional': 81, 'Renaissance': 82, 'Ri

In [29]:
#encode decode

In [31]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int=vocab
        self.int_to_str= {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed= [item.strip() for item in preprocessed if item.strip()]
        ids= [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text= " ".join([self.int_to_str[i] for i in ids])
        #replace space before the specified punctuations
        text= re.sub(r'\s+([,.?!"()\'])',r'\1', text)
        return text

In [33]:
tokenizer = SimpleTokenizerV1(vocab)

text= """"It's the last he painted, you know," 
          Mrs. Gisburn said with pardonable pride."""
ids= tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [35]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [37]:
#test with unknown data not available in vocab

In [39]:
text= """hello I love tea."""
ids=tokenizer.encode(text)

KeyError: 'hello'

In [41]:
# add 2 new tokens to handle unkown token and end of document

In [43]:
all_tokens= sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab={token:integer for integer, token in enumerate(all_tokens)}


In [45]:
len(vocab.items())

1132

In [47]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [49]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int=vocab
        self.int_to_str= {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed= [item.strip() for item in preprocessed if item.strip()]
        
        preprocessed=[ 
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        
        ids= [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text= " ".join([self.int_to_str[i] for i in ids])
        #replace space before the specified punctuations
        text= re.sub(r'\s+([,.?!"()\'])',r'\1', text)
        return text

In [51]:
tokenizer = SimpleTokenizerV2(vocab)

text1="Hello, do you like tea?"
text2="In the sunlit terrace of the palace."

text=" <|endoftext|> ".join((text1,text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terrace of the palace.


In [53]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 983, 722, 988, 1131, 7]

In [55]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terrace of the <|unk|>.'

## BPE - Byte Pair Encoding

In [76]:
! pip3 install tiktoken


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [77]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.11.0


In [78]:
tokenizer= tiktoken.get_encoding("gpt2")

In [79]:
text=("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace.")

integers= tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [80]:
# convert them back to text from ids

In [81]:
string = tokenizer.decode(integers)
print(string)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [82]:
# GPT 2 and GPT3 have a total vocabulary size of 50256, less than their total words, which is more 
# than 200000 because of BPE
#The algorithm underlying BPE breaks down words that aren't in its predefined vocabulary into smaller subword units of even individual characters.    
#The enables it to handle out-ofvocabulary words.
#So, thanks to the BPE algorithm, if the tokenizer encounters an unfamiliar word during tokenization, it can represent it as sequence of subword tokens or characters

In [83]:
 integers= tokenizer.encode("Akwirw ier")
print(integers)

string = tokenizer.decode(integers)
print (string)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


## Creating Input Target pairs.

In [85]:
#we will do data loader and ferch input-target pairs using sliding window

In [86]:
with open("Data/the-verdict.txt","r", encoding="utf-8") as f:
    raw_text=f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [98]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [121]:
enc_sample= enc_text :]

In [ ]:
#One of the easiest and most intuitive ways to create the input-target pairs for the nextword prediction task is to create two variables, x and y, where x contains the
#input tokens and y contains the targets, which are the inputs shifted by 1:

In [123]:
context_size= 4  # length of input
#The context_size of 4 means that the model is trained to look at a sequence of 4 words (or tokens)
#to predict the next word in the sequence.
#The input x is the first 4 tokens [1, 2, 3, 4], and the target y is the next 4 tokens [2, 3, 4, 5|

x= enc_sample[:context_size]
y= enc_sample[1:context_size+1]

print(f"x:  {x}")
print(f"y:  {y}")


x:  [290, 4920, 2241, 287]
y:  [4920, 2241, 287, 257]


In [ ]:
#Processing the inputs along with the targets, which are the inputs shifted by one position, we can then create the next-word prediction tasks as follows:

In [125]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired= enc_sample[i]

    print(context, "----->",desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [ ]:
#Everything left of the arrow (--->) refers to the input an LLM would receive, and the token ID

In [131]:
for i in range(1, context_size+1):
    context=enc_sample[:i]
    desired=enc_sample[i]
    print(tokenizer.decode(context),"------>", tokenizer.decode([desired]))

 and ------>  established
 and established ------>  himself
 and established himself ------>  in
 and established himself in ------>  a


In [ ]:
# we need to implement data loader for more efficient with pyTorch 

In [135]:
pip install torch

  Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl.metadata (25 kB)
Using cached torch-2.2.2-cp312-none-macosx_10_9_x86_64.whl (150.8 MB)

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [159]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]

        #Tokenize the entire text
        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

        #use a sliding window to chunk the book into overlapping sequence of max_length
        # sliding window is take entire row or max_length of texts and then slide next row

        for i in range(0, len(token_ids)- max_length, stride):
            input_chunk= token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [161]:
def create_dataloader_V1(text, batch_size=4, max_length=256, stride=128, shuffle=True,
                         drop_last=True, num_workers=0):
    #initaalize the tokenizer
    tokenizer= tiktoken.get_encoding("gpt2")

    #Create dataSet
    dataset= GPTDatasetV1(text,tokenizer,max_length,stride)

    #Create dataloader
    dataloader= DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader
        

In [155]:
with open("Data/the-verdict.txt","r", encoding="utf-8") as f:
    raw_text=f.read()

In [163]:
import torch
print("PyTorch version:", torch.__version__)

dataloader= create_dataloader_V1(raw_text, batch_size=1, max_length=4,stride=1,shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print (first_batch)

PyTorch version: 2.2.2
[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [165]:
# stride means  you shift cells of input in each iteration: if it's 1 it will only shift 1 cell 
# if it's 4 it will shift 4 cells and then give input from 5th cell.

In [167]:
second_batch= next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
#let's have a brief Iook how we can use the data loader to sample with a batch size greater than 1:

In [173]:
dataloader= create_dataloader_V1(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)
data_iter=iter(dataloader)
inputs,targets=next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n",targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
